# Programming Assignment 2: Movie recommendation

- 과제 목표: 뉴럴 네트워크 모델을 설계한 후 모델을 학습하여 각 영화들의 embedding 들을 생성하고, 영화 embedding 을 활용하여 각 사용자에게 맞춤형 영화를 추천

# Notice

<br>

- 과제를 수행하면서 각 task 마다 꼭 주어진 1개의 cell만을 사용할 필요는 없으며, 여러 개의 cell을 추가하여 자유롭게 사용해도 괜찮습니다.
- 과제 수행을 위해 필요한 module이 있다면 추가로 import 해도 괜찮습니다.

# Import Modules

In [1]:
import warnings, random
import numpy as np
import pandas as pd
import torch # For building network

from itertools import permutations # For making pairs

warnings.filterwarnings('ignore')

# Data loading

In [2]:
dir = './MovieLens100K/'
df_ratings = pd.read_csv(dir + 'ratings.csv', usecols=['userId', 'movieId', 'rating'])
df_movies = pd.read_csv(dir + 'movies.csv', usecols=['movieId', 'title', 'genres']) # for title-matching

# Preprocessing data

<br>

> ### Problem 1 (3 points)

<br>

1. df_ratings로 부터 각 사용자들이 본 영화를 기록.
2. 사용자 마다 본 영화 목록을 $(movie1, movie2)$, $(movie2, movie1)$ 과 같이 pair로 생성.
    - 즉, 각 사용자 마다 본 영화 목록에 대해 Permutation을 수행
3. 2번 과정이 끝난 후, random을 이용해 각 pair 순서를 무작위로 shuffle.

In [3]:
#### Your Code Here
df_users = df_ratings['userId'].drop_duplicates()
movies_permutation_result = []
cnt = 0
for i in df_users:
    if cnt == 20:
        break
    is_user_movies = df_ratings['userId'] == i
    user_movies = df_ratings[is_user_movies]['movieId']
    user_movies_list = user_movies.values.tolist()
    user_movies_permutation = list(permutations(user_movies_list, 2))
    movies_permutation_result.extend(user_movies_permutation)
    cnt += 1
random.shuffle(movies_permutation_result)
# print(len(movies_permutation_result))

In [4]:
len(movies_permutation_result)

1099200

In [5]:
print(movies_permutation_result[:10])

[(1441, 1906), (10, 1895), (2840, 1892), (79132, 1203), (2485, 2065), (64839, 750), (2143, 1025), (587, 4896), (378, 3081), (1805, 3591)]


# Build and train neural networks for generating movie embeddings

<br>

> ### Problem 2 (4 points)

<br>

- 각 영화 임베딩을 구하기 위해 뉴럴 네트워크 모델을 활용하여 multi-class classification 을 수행

- 설계할 신경망의 기본 구조는 **Input Layer - Hidden(Embedding) Layer - Output Layer**.
    - 7주차 강의자료 p.7 신경망 구조 이미지 참고

- 현재 Network를 통해 하고자 하는 task는 multi-class classification.
    - 예: $(movie1, movie2)$ 와 같은 입력 데이터와 정답 출력 데이터를 이용해 모델을 학습
        - Input : $movie1$의 one-hot vector
        - Output : $\widehat{movie2}$의 one-hot vector
        - Compute Loss : $\widehat{movie2}$ 와 $movie2$ 간의 Cross-entropy Loss
- 학습이 완료된 이후에 input layer와 hidden(embedding) layer 사이의 weight matrix $W_{in}$를 movie에 대한 embedding vector로 사용이 가능.
> embedding size(# of hidden units)는 100 이하로 두는 것을 권장. <br>
> embedding layer 다음 hidden layer를 더 추가하여 Genre와 같은 추가 정보를 학습에 활용 할 수도 있음 (필수적으로 고려해야할 사항은 아님).

- 설계한 뉴럴 네트워크 모델의 학습이 완료된 후, 학습된 weight matrix $W_{in}$의 행/열벡터를 각 영화에 대한 embedding vector로 간주하여 영화 embedding 들을 구할 수 있음.

In [6]:
import torch.nn as nn
class NNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.input_layer = nn.Linear(input_size, 100)
        self.hidden_layer = nn.Linear(100, 100)
        self.output_layer = nn.Linear(100, input_size)
        self.relu = nn.ReLU()
        self.activation_func = nn.Sigmoid()
        self.loss_func = nn.CrossEntropyLoss()
    def forward(self, x, target):
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.hidden_layer(x)
        x = self.relu(x)
        x = self.output_layer(x)
        output = self.activation_func(x)
        print(output.shape)
        print(target.shape)
        loss = self.loss_func(output[0], target[0])
        return loss

In [7]:
#### Your Code Here
#[(6934, 2641), (153, 7153), (1278, 260), (130450, 3764), (541, 1997), (49530, 1037), (116823, 117849), (459, 2443), (4255, 3174), (102602, 141810)]
#(data, target)

#unique mapping
tensor_movie = torch.as_tensor(movies_permutation_result)
print(tensor_movie.shape)
print(tensor_movie[0:4])
tensor_movie1 = tensor_movie[:,0]
movie1_unique = tensor_movie1.unique() #mapping array
unique_len = movie1_unique.shape[0]
print(movie1_unique.shape) #1859
tensor_movie2 = tensor_movie[:,1]
movie2_unique = tensor_movie2.unique()
print(movie2_unique.shape) #1859
model = NNet(unique_len)
#for epoch in range(100):
for input_data, target in tensor_movie: #1개의 pair
    #make one_hot vector (input, target generation)
    input_tensor = torch.zeros((1, unique_len)) # B x 1859
    input_tensor[0][(movie1_unique == input_data).nonzero(as_tuple=True)[0]] = 1
    target_tensor = torch.zeros((1, unique_len))
    target_tensor[0][(movie2_unique == target).nonzero(as_tuple=True)[0]] = 1
    
    #model forwarding
    loss = model(input_tensor, target_tensor)
    print(loss)
    break
    #loss function
    
    
        
        


torch.Size([1099200, 2])
tensor([[ 1441,  1906],
        [   10,  1895],
        [ 2840,  1892],
        [79132,  1203]])
torch.Size([1859])
torch.Size([1859])
torch.Size([1, 1859])
torch.Size([1, 1859])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

# Recommend customized movies to user

<br>

> ### Problem 3 (3 points)

<br>

- 임의의 한명의 사용자에 대하여 해당 사용자가 봤던 영화 n개에 대해 **통합된 embedding vector**를 생성.
    - n개의 embedding vector들에 대해, element-wise한 계산을 통해 통합된 하나의 embedding vector를 생성.
    - 이 embedding vector는 해당 사용자의 전반적인 영화 시청 성향을 나타내는 embedding vector로 간주할 수 있음.
    - 즉, **사용자 1명 당 1개의 embedding vector**를 가짐.
- 통합된 embedding vector와 학습된 weight matrix $W_{in}$의 모든 영화 embedding vector들 간의 유사도를 계산.
- 그 중 유사도가 높은 (top n) 영화들을 선정, 사용자에게 추천.
    > Recommended format : MovieId, Title, Genre, Similarity 가 포함된 형식

In [ ]:
#### Your Code Here
